# hyprovo: A Minimal Testing Framework for Hy

Before running tests, the `setup-test-env` macro must be called to setup the parameters of the test environment. 

In [1]:
(require [hyprovo.framework [setup-test-env]])

(setup-test-env)

This is required if tests are run in a global environment. Tests can also be executed within *suites* which automatically take care of initializing the (local) test environment. Suites are the recommended way to run tests (see the section on [Suites](#Suites) for details).

## Output and Logging

All output is logged using the builtin [logging](https://docs.python.org/3/library/logging.html) module. A logger module, `hyprovo.logger`, is provided that offers three frequently used loggers,

* `console-logger`
* `file-logger`
* `console-and-file-logger`

The last two are functions that require a `:log-file` (filename) argument.

Logging is set using the `*test-logger*` variable. `setup-test-env` initializes `*test-logger*`. By default, the `*test-logger*` variable is set to `hyprovo.logger.console-logger`. 

`*test-logger*` can be set to any custom logger (see the `hyprovo.logger` module for examples).

## Validation and Reporting

The `check` macro is the basic "unit" of validation. It takes one or more statements of the form,

```
(op expr expected description)
```

where `op` is the comparison (operator), `expr` is the expression to be tested, `expected` is the result to compare against and `description` is a string describing the test case. All arguments are mandatory. As an example,

In [2]:
(require [hyprovo.framework [check]])

(check 
   (= (+ 1 2) 3 "Add pass")
   (= (+ 1 2) 5 "Add fail"))

2021-11-23 16:08:38.043 - INFO - Pass ✓...  Add pass
2021-11-23 16:08:38.050 - ERROR - Fail ✗...  Add fail 
                                    Expected: 5
                                    Got: 3


(2, 1, 1)

Note above that the logging output is formatted as,

```
Timestamp - Level - Message
```

For tests that fail, both the expected and actual values are printed.

The tuple `(2, 1, 1)` is the return value of `check` denoting the aggregated statistics for all the test results. It is of the form,

```
(total, pass, fail)
```

So, `(2, 1, 1)` indicates that out of a total of 2 tests, 1 passed and 1 failed.

## Combining Results

The `combine-results` function is useful for aggregating the `(total, pass, fail)` statistics from multiple tests. For example, the two statements in `check` above could be written separately and then combined as follows,

In [3]:
(import [hyprovo.framework [combine-results]])

(combine-results 
 (check (= (+ 1 2) 3 "Add pass"))
 (check (= (+ 1 2) 5 "Add fail")))

2021-11-23 16:08:38.093 - INFO - Pass ✓...  Add pass
2021-11-23 16:08:38.104 - ERROR - Fail ✗...  Add fail 
                                    Expected: 5
                                    Got: 3


(2, 1, 1)

The `report-combined-results` macro is useful to log the aggregate summary `(total, pass, fail)` tuple in textual form. This macro returns the `(total, pass, fail)` tuple so it can be used to report intermediate results,

In [4]:
(require [hyprovo.framework [report-combined-results]])

(-> (combine-results
     (-> (combine-results 
          (check (= (+ 1 2) 3 "Add pass"))
          (check (= (+ 1 2) 5 "Add fail")))
     (report-combined-results))
     (check (= (- 1 2) -1 "Sub pass"))))

2021-11-23 16:08:38.184 - INFO - Pass ✓...  Add pass
2021-11-23 16:08:38.195 - ERROR - Fail ✗...  Add fail 
                                    Expected: 5
                                    Got: 3
2021-11-23 16:08:38.196 - DEBUG -  Test Summary → Total: 2    Pass: 1    Fail: 1
2021-11-23 16:08:38.213 - INFO - Pass ✓...  Sub pass


(3, 2, 1)

The `Test Summary` line is added by `report-combined-results`.

## Defining Tests

Tests are defined using the `deftest` macro. `deftest` is typically used to group related tests under a common name. It may also contain parameter declarations required to run the tests. 

The `check` statement presented previously can be wrapped in `deftest` as,

In [5]:
(require [hyprovo.framework [deftest]])

(deftest test-add
  (check 
   (= (+ 1 2) 3 "Add pass")
   (= (+ 1 2) 5 "Add fail")))

<function test_add at 0x7fe584daab70>

The `deftest` macro returns a macro that returns an anonymous function. To execute a test defined using `deftest`, use the "double parenthesis" notation, 

In [6]:
((test-add))

2021-11-23 16:08:38.432 - INFO - Pass ✓... (test-add): Add pass
2021-11-23 16:08:38.439 - ERROR - Fail ✗... (test-add): Add fail 
                                    Expected: 5
                                    Got: 3


(2, 1, 1)

The inner most set of parenthesis defines the function, while the outermost executes it. This construct thus allows simultaneous definition and execution of the test function at the point of call. 

> The name of the test function is added to the output log statement.

The test body may contain any statement. It is suggested that the last statement return the test aggregate `(total, pass, fail)` tuple so that results can be combined with other test functions,

In [7]:
(deftest test-add-var
  (setv a 1 b 2)
  (check 
   (= (+ a b) 3 "Add pass")
   (= (+ a b) 5 "Add fail")))

((test-add-var))

2021-11-23 16:08:38.687 - INFO - Pass ✓... (test-add-var): Add pass
2021-11-23 16:08:38.693 - ERROR - Fail ✗... (test-add-var): Add fail 
                                    Expected: 5
                                    Got: 3


(2, 1, 1)

For many tests that require a common set of setup and teardown operations, consider using *fixtures* described in the next section.

Tests can be nested, and also combined with check statements,

In [8]:
(deftest test-sub
  (check 
   (= (- 1 2) -1 "Sub pass")
   (= (- 1 2) 1 "Sub fail")))

<function test_sub at 0x7fe584bb29d8>

In [9]:
(deftest test-basic-arithmetic
         (setv a 2 b 2)
         (-> (combine-results 
              ((test-add)) 
              ((test-sub))
              (check (= (+ a b) 4 "Add pass")))
             (report-combined-results)))

((test-basic-arithmetic))

2021-11-23 16:08:39.362 - INFO - Pass ✓... (test-basic-arithmetic): Add pass
2021-11-23 16:08:39.374 - INFO - Pass ✓... (test-basic-arithmetic / test-add): Add pass
2021-11-23 16:08:39.381 - ERROR - Fail ✗... (test-basic-arithmetic / test-add): Add fail 
                                    Expected: 5
                                    Got: 3
2021-11-23 16:08:39.397 - INFO - Pass ✓... (test-basic-arithmetic / test-sub): Sub pass
2021-11-23 16:08:39.405 - ERROR - Fail ✗... (test-basic-arithmetic / test-sub): Sub fail 
                                    Expected: 1
                                    Got: -1
2021-11-23 16:08:39.406 - DEBUG - (test-basic-arithmetic): Test Summary → Total: 5    Pass: 3    Fail: 2


(5, 3, 2)

> For nested functions, the names of each of the outer test functions are added to the path of the test name.

The function `run-tests` can also be used to execute tests. It accepts multiple tests as arguments and provides a cleaner interface. `▶` is an alias to `run-tests`. Using this, the `test-basic-arithmetic` function could be written more simply,

In [10]:
(import [hyprovo.framework [▶]])

(deftest test-basic-arithmetic
         (setv a 2 b 2)
         (-> (combine-results 
              (▶ (test-add) (test-sub))
              (check (= (+ a b) 4 "Add pass")))
             (report-combined-results)))

((test-basic-arithmetic))

2021-11-23 16:08:39.888 - INFO - Pass ✓... (test-basic-arithmetic): Add pass
2021-11-23 16:08:39.903 - INFO - Pass ✓... (test-basic-arithmetic / test-add): Add pass
2021-11-23 16:08:39.912 - ERROR - Fail ✗... (test-basic-arithmetic / test-add): Add fail 
                                    Expected: 5
                                    Got: 3
2021-11-23 16:08:39.928 - INFO - Pass ✓... (test-basic-arithmetic / test-sub): Sub pass
2021-11-23 16:08:39.937 - ERROR - Fail ✗... (test-basic-arithmetic / test-sub): Sub fail 
                                    Expected: 1
                                    Got: -1
2021-11-23 16:08:39.938 - DEBUG - (test-basic-arithmetic): Test Summary → Total: 5    Pass: 3    Fail: 2


(5, 3, 2)

## Fixtures

[Fixtures](https://stackoverflow.com/questions/12071344/what-are-fixtures-in-programming) are defined in terms of setup and teardown operations using the `deffixture` macro and take the following form,

```
(deffixture fixture-name setup teardown)
```

In [11]:
(require [hyprovo.framework [deffixture]])

(deffixture fix-a ((setv a 1 b 1)) ((del a b)))

<function fix_a at 0x7fe584daae18>

defines a (hypothetical) fixture `fix-a` that sets values of `a` and `b` in the setup, and deletes them in the teardown (a more realistic use case might involve, say, connecting to a database (setup) and closing those connections (teardown)). Use the `with-fixture` macro to apply this fixture,

In [12]:
(deftest test-add-fixvar
         (check
          (= (+ a b) 2 "Add pass")
          (= (+ a b) 5 "Add fail")))

<function test_add_fixvar at 0x7fe584daa730>

In [13]:
(require [hyprovo.framework [with-fixture]])

(▶ (with-fixture fix-a (▶ (test-add-fixvar))))

2021-11-23 16:08:40.204 - INFO - Pass ✓... (test-add-fixvar): Add pass
2021-11-23 16:08:40.214 - ERROR - Fail ✗... (test-add-fixvar): Add fail 
                                    Expected: 5
                                    Got: 2


(2, 1, 1)

Multiple tests can be run using the same fixture,

In [14]:
(deftest test-sub-fixvar
         (check
          (= (- a b) 0 "Sub pass")
          (= (- a b) -1 "Sub fail")))

<function test_sub_fixvar at 0x7fe584bb2bf8>

In [15]:
(▶ (with-fixture fix-a (▶ (test-add-fixvar) (test-sub-fixvar))))

2021-11-23 16:08:40.523 - INFO - Pass ✓... (test-add-fixvar): Add pass
2021-11-23 16:08:40.532 - ERROR - Fail ✗... (test-add-fixvar): Add fail 
                                    Expected: 5
                                    Got: 2
2021-11-23 16:08:40.547 - INFO - Pass ✓... (test-sub-fixvar): Sub pass
2021-11-23 16:08:40.558 - ERROR - Fail ✗... (test-sub-fixvar): Sub fail 
                                    Expected: -1
                                    Got: 0


(4, 2, 2)

Variable declarations inside the test body take precedence over those defined in the fixture, consistent with scoping rules (LEGB),

In [16]:
(▶ (with-fixture fix-a (▶ (test-add-var))))

2021-11-23 16:08:40.638 - INFO - Pass ✓... (test-add-var): Add pass
2021-11-23 16:08:40.646 - ERROR - Fail ✗... (test-add-var): Add fail 
                                    Expected: 5
                                    Got: 3


(2, 1, 1)

Fixtures can be nested,

In [17]:
(deffixture fix-c ((setv a 1)) (None))
(deffixture fix-d ((setv b 1)) (None))

(▶ (with-fixture fix-c 
    (▶ (with-fixture fix-d 
        (▶ (test-add-var))))))

2021-11-23 16:08:40.765 - INFO - Pass ✓... (test-add-var): Add pass
2021-11-23 16:08:40.774 - ERROR - Fail ✗... (test-add-var): Add fail 
                                    Expected: 5
                                    Got: 3


(2, 1, 1)

Use `with-fixtures` to run multiple tests with multiple fixtures,

In [18]:
(require [hyprovo.framework [with-fixtures]])

(deffixture fix-b ((setv a 1) (setv b 2)) ((del a) (del b)))

<function fix_b at 0x7fe584c3f378>

(`fix-b` illustrates a fixture definition with multiple setup and teardown statements.)

In [19]:
(▶ (with-fixtures [fix-a fix-b] (▶ (test-add-fixvar) (test-sub-fixvar))))

2021-11-23 16:08:41.052 - INFO - Pass ✓... (test-add-fixvar): Add pass
2021-11-23 16:08:41.067 - ERROR - Fail ✗... (test-add-fixvar): Add fail 
                                    Expected: 5
                                    Got: 2
2021-11-23 16:08:41.089 - INFO - Pass ✓... (test-sub-fixvar): Sub pass
2021-11-23 16:08:41.099 - ERROR - Fail ✗... (test-sub-fixvar): Sub fail 
                                    Expected: -1
                                    Got: 0
2021-11-23 16:08:41.120 - ERROR - Fail ✗... (test-add-fixvar): Add pass 
                                    Expected: 2
                                    Got: 3
2021-11-23 16:08:41.132 - ERROR - Fail ✗... (test-add-fixvar): Add fail 
                                    Expected: 5
                                    Got: 3
2021-11-23 16:08:41.152 - ERROR - Fail ✗... (test-sub-fixvar): Sub pass 
                                    Expected: 0
                                    Got: -1
2021-11-23 16:08:41.167 - INFO - Pas

(8, 3, 5)

The first argument is a list of fixtures followed by the tests. All tests are run with each fixture in the list. 

Fixtures can be defined inside or outside tests.

## Suites

Suites are the recommended way to run tests, checks and fixtures since they automatically take care of setting up the test environment (`setup-test-env` is not required). There is typically one suite per module.

The `defsuite` macro is used to define a suite,

In [20]:
(require [hyprovo.framework [defsuite]])

(defsuite test-suite-arithmetic-1
   (-> (combine-results (▶ (test-add) (test-sub)))
       (report-combined-results)))

`defsuite` returns a function with the suite name that takes an optional `test-logger` argument (defaults to `hyprovo.logger.console-logger`). To run the suite,

In [21]:
(test-suite-arithmetic-1)

2021-11-23 16:08:41.337 - INFO - Pass ✓... (test-suite-arithmetic-1 / test-add): Add pass
2021-11-23 16:08:41.344 - ERROR - Fail ✗... (test-suite-arithmetic-1 / test-add): Add fail 
                                    Expected: 5
                                    Got: 3
2021-11-23 16:08:41.361 - INFO - Pass ✓... (test-suite-arithmetic-1 / test-sub): Sub pass
2021-11-23 16:08:41.369 - ERROR - Fail ✗... (test-suite-arithmetic-1 / test-sub): Sub fail 
                                    Expected: 1
                                    Got: -1
2021-11-23 16:08:41.370 - DEBUG - (test-suite-arithmetic-1): Test Summary → Total: 4    Pass: 2    Fail: 2


(4, 2, 2)

> The name of the suite is added to the path of the test name.

Suites can be nested. To use the same logger across all suites provide the same `test-logger` argument,

In [22]:
(defsuite test-suite-arithmetic-2
   (-> (combine-results 
        (check
          (= (* 1 2) 2 "Mul pass")
          (= (* 1 2) -1 "Mul fail"))
          (▶ (with-fixture fix-a (▶ (test-add-fixvar)))))
       (report-combined-results)))

In [23]:
(defsuite test-suite-arithmetic
    (-> (combine-results 
         (test-suite-arithmetic-1 :test-logger test-logger)
         (test-suite-arithmetic-2 :test-logger test-logger))
        (report-combined-results)))

(test-suite-arithmetic :test-logger console-logger)

2021-11-23 16:08:41.561 - INFO - Pass ✓... (test-suite-arithmetic-1 / test-add): Add pass
2021-11-23 16:08:41.570 - ERROR - Fail ✗... (test-suite-arithmetic-1 / test-add): Add fail 
                                    Expected: 5
                                    Got: 3
2021-11-23 16:08:41.587 - INFO - Pass ✓... (test-suite-arithmetic-1 / test-sub): Sub pass
2021-11-23 16:08:41.595 - ERROR - Fail ✗... (test-suite-arithmetic-1 / test-sub): Sub fail 
                                    Expected: 1
                                    Got: -1
2021-11-23 16:08:41.596 - DEBUG - (test-suite-arithmetic-1): Test Summary → Total: 4    Pass: 2    Fail: 2
2021-11-23 16:08:41.616 - INFO - Pass ✓... (test-suite-arithmetic-2): Mul pass
2021-11-23 16:08:41.624 - ERROR - Fail ✗... (test-suite-arithmetic-2): Mul fail 
                                    Expected: -1
                                    Got: 2
2021-11-23 16:08:41.649 - INFO - Pass ✓... (test-suite-arithmetic-2 / test-add-fixvar): Add pa

(8, 4, 4)

> For nested suites, the name of the outer suites are ***not*** added to the path of the test name.

## More Examples
This section describes various utilities available for writing tests. For further examples see [cases.hy](https://github.com/prasxanth/hyprovo/blob/main/tests/cases.hy).